In [1]:
# @title Установка и импорты
!pip install openai langchain langchain_openai langchain_community tiktoken faiss-cpu pytz openpyxl fuzzywuzzy transliterate python-Levenshtein
!pip install nest_asyncio

from IPython.core.display import clear_output
from IPython.display import HTML, display
clear_output()
import openpyxl
import os
import getpass
import openai
import re
import time
import pytz
from datetime import datetime
from fuzzywuzzy import fuzz, process
from transliterate import translit
import asyncio
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from openai import AsyncOpenAI
from oauth2client.service_account import ServiceAccountCredentials

from google.colab import drive, userdata
drive.mount('/content/drive', force_remount=True)

try:
  openai_key = userdata.get('OPENAI_API_KEY')
except Exception as e:
  openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key

client = AsyncOpenAI()


# Цветной print
def print_color(text, color='black'):
    color_codes = {
        'black': '#000000',
        'red': '#FF4136',
        'green': '#2ECC40',
        'blue': '#0074D9',
        'yellow': '#FFDC00',
        'purple': '#B10DC9',
        'cyan': '#7FDBFF',
        'white': '#FFFFFF',
        'orange': '#FF851B',
        'pink': '#FF69B4'
    }

    color_code = color_codes.get(color.lower(), color)
    display(HTML(f'<text style="color:{color_code}">{text}</text>'))

# Функция переноса теста в ячейках
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)


def current_datetime():
    # Получаем текущую дату и время во временной зоне Москва
    moscow_tz = pytz.timezone('Europe/Moscow')
    current_time = datetime.now(moscow_tz)

    # Форматируем текущую дату и время в требуемые форматы
    formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")

    return formatted_time


import os.path
import tempfile
from typing import List, Set, Dict

import requests
import gspread
from oauth2client.service_account import ServiceAccountCredentials



class GoogleSheetHandler:
    """
    A handler for interacting with Google Sheets.

    This class provides methods for authenticating with Google Sheets API
    and manipulating data in a specified worksheet.

    Attributes:
        credentials (ServiceAccountCredentials): The credentials for accessing Google Sheets API.
        gc (gspread.client.Client): The gspread client instance for interacting with Google Sheets.
        spreadsheet_url (str): The URL of the Google Sheets spreadsheet.
    """

    def __init__(self, json_url: str, spreadsheet_url: str) -> None:
        """
        Initializes the GoogleSheetHandler with the given credentials and spreadsheet URL.

        Args:
            json_url (str): The URL to the JSON file containing the service account credentials.
            spreadsheet_url (str): The URL of the Google Sheets spreadsheet to interact with.
        """
        self.json_url = json_url
        self.spreadsheet_url = spreadsheet_url
        self.credentials = self._load_credentials(self.json_url)
        self.gc = gspread.authorize(self.credentials)


    def _load_credentials(self, json_url: str) -> ServiceAccountCredentials:
        """
        Loads service account credentials from a given URL.

        Args:
            json_url (str): The URL to the JSON file containing the service account credentials.

        Returns:
            ServiceAccountCredentials: The credentials for accessing Google Sheets API.

        Raises:
            ValueError: If the response content is not valid JSON.
            requests.RequestException: If there's an error while making the HTTP request.
        """
        try:
            response = requests.get(json_url)
            response.raise_for_status()  # Raise an exception for bad status codes (e.g., 404, 500)
            response_json = response.json()

            credentials = ServiceAccountCredentials.from_json_keyfile_dict(response_json, ['https://spreadsheets.google.com/feeds'])
        except requests.RequestException as e:
            raise e  # Re-raise the exception
        except ValueError as e:
            raise ValueError("Invalid JSON format in the response.") from e
        finally:
            if os.path.exists('credentials.json'):
                os.remove('credentials.json')

        return credentials

    def append_row(self, data_to_append: List[str], worksheet_name: str) -> None:
        """
        Appends a row of data to the bottom of the specified worksheet.

        Args:
            data_to_append (List[str]): The data to append to the worksheet.
            worksheet_name (str): The name of the worksheet to append the data to.
        """
        worksheet = self.gc.open_by_url(self.spreadsheet_url).worksheet(worksheet_name)
        worksheet.append_row(data_to_append)

    def insert_row(self, data_to_insert: List[str], index: int, worksheet_name: str) -> None:
        """
        Inserts a row of data at a specified index in the worksheet.

        Args:
            data_to_insert (List[str]): The data to insert into the worksheet.
            index (int): The position at which to insert the new row.
            worksheet_name (str): The name of the worksheet to insert the data into.
        """
        worksheet = self.gc.open_by_url(self.spreadsheet_url).worksheet(worksheet_name)
        worksheet.insert_row(data_to_insert, index)

    def append_row_to_top(self, data_to_append: List[str], worksheet_name: str) -> None:
        """
        Appends a row of data to the top of the specified worksheet.

        Args:
            data_to_append (List[str]): The data to append to the top of the worksheet.
            worksheet_name (str): The name of the worksheet to append the data to.
        """
        self.insert_row(data_to_append, 1, worksheet_name)

    def update_cell(self, row: int, col: int, value: str, worksheet_name: str) -> None:
        """
        Updates the value of a specific cell in the worksheet.

        Args:
            row (int): The row number of the cell to update.
            col (int): The column number of the cell to update.
            value (str): The new value for the cell.
            worksheet_name (str): The name of the worksheet containing the cell.
        """
        worksheet = self.gc.open_by_url(self.spreadsheet_url).worksheet(worksheet_name)
        worksheet.update_cell(row, col, value)

    def delete_row(self, index: int, worksheet_name: str) -> None:
        """
        Deletes a specific row in the worksheet.

        Args:
            index (int): The position of the row to delete.
            worksheet_name (str): The name of the worksheet to delete the row from.
        """
        worksheet = self.gc.open_by_url(self.spreadsheet_url).worksheet(worksheet_name)
        worksheet.delete_rows(index)

    def read_data(self, cell_range: str, worksheet_name: str) -> List[List[str]]:
        """
        Reads data from a specific range in the worksheet.

        Args:
            cell_range (str): The range of cells to read (e.g., 'A1:C10').
            worksheet_name (str): The name of the worksheet to read the data from.

        Returns:
            List[List[str]]: The data read from the specified range.
        """
        worksheet = self.gc.open_by_url(self.spreadsheet_url).worksheet(worksheet_name)
        return worksheet.get(cell_range)

    def clear_range(self, cell_range: str, worksheet_name: str) -> None:
        """
        Clears data from a specific range in the worksheet.

        Args:
            cell_range (str): The range of cells to clear (e.g., 'A1:C10').
            worksheet_name (str): The name of the worksheet to clear the data from.
        """
        worksheet = self.gc.open_by_url(self.spreadsheet_url).worksheet(worksheet_name)
        worksheet.batch_clear([cell_range])


    def append_rows(self, data_to_append: List[List[str]], worksheet_name: str) -> None:
        """
        Appends multiple rows of data to the bottom of the specified worksheet in a single API call.

        Args:
            data_to_append (List[List[str]]): The data to append to the worksheet.
            worksheet_name (str): The name of the worksheet to append the data to.
        """
        worksheet = self.gc.open_by_url(self.spreadsheet_url).worksheet(worksheet_name)
        worksheet.append_rows(data_to_append)


spreadsheet_url='https://docs.google.com/spreadsheets/d/1607FPIz2-SOwR31rPYlFkYrOo37q9rC9xixcoEo5G8E/edit?usp=sharing'
gs_handler = GoogleSheetHandler(
    json_url='https://drive.usercontent.google.com/u/0/uc?id=18sDUm52vPA6mCJgocqLZLiPc0PNTsAfS&export=download',
    spreadsheet_url=spreadsheet_url,
)

questions_spredsheet_url = 'https://docs.google.com/spreadsheets/d/1rm9bvjDFkRuyU94Lp1A5L4GlzUNkNt7nAA-mmWYf4TY/edit?usp=sharing'
gs_questions = GoogleSheetHandler(
    json_url='https://drive.usercontent.google.com/u/0/uc?id=18sDUm52vPA6mCJgocqLZLiPc0PNTsAfS&export=download',
    spreadsheet_url=questions_spredsheet_url,
)


from functools import cached_property


class Scenario:
    def __init__(self, name: str,  group: str):
        self.name = name
        self.group = group


class KeyWordsCollector:
    def __init__(self, xlsx_path: str):
        self.xlsx_path = xlsx_path

    @cached_property
    def scenarios(self) -> List[Scenario]:
        workbook = openpyxl.load_workbook(self.xlsx_path)
        sheet = workbook.active
        scenarios: List[Scenario] = []

        for row in sheet.iter_rows(min_row=1, max_row=sheet.max_row, values_only=True):
            scenario_name = row[1].lower()
            scenario_group = row[5]
            scenario = Scenario(scenario_name, scenario_group)
            scenarios.append(scenario)

        return scenarios

    @staticmethod
    def clean_non_unique_key_words(key_word_sets: Dict[str, Set[str]]) -> None:
        # Step 1: Count the occurrences of each name
        name_count = {}

        for set_name, names in key_word_sets.items():
            for name in names:
                if name in name_count:
                    name_count[name] += 1
                else:
                    name_count[name] = 1

        # Step 2: Identify names that are not unique (appear in more than one set)
        non_unique_names = {name for name, count in name_count.items() if count > 1}

        # Step 3: Remove non-unique names from each set
        for set_name, names in key_word_sets.items():
            key_word_sets[set_name] = names - non_unique_names

    @cached_property
    def key_words_sets(self) -> Dict[str, Set[str]]:
        result = {}
        for scenario in self.scenarios:
            if scenario.group not in result:
                result[scenario.group] = set()

            key_word: str = scenario.name.split(']')[0][1:]
            result[scenario.group].add(key_word)

        self.clean_non_unique_key_words(result)
        return result

    def find_relevant_group(self, user_question:str) -> str | None:
        question = user_question.lower()
        user_words = user_question.lower().replace(',', '').replace('.','').split(' ')

        result = ''
        total_count = 0

        for group_name, key_words in self.key_words_sets.items():
            for word in user_words:
                if word in key_words:
                    total_count += 1
                    if total_count > 1:
                        return None
                    result = group_name

        return result


def get_tags_from_xlsx(xlsx_path: str, sheet_name='Теги') -> Dict[str, str]:
    wb = openpyxl.load_workbook(xlsx_path)
    sheet = wb[sheet_name]
    tags = {}

    for row in sheet.iter_rows(values_only=True, min_row=3, max_row=546):
        if row[1] is not None:
            translited_tag = translit(row[1].lower(), 'ru', reversed=True)
            tags[translited_tag] = row[1]

    return tags


Mounted at /content/drive


In [ ]:
# @title Загрузка эксель файла
xlsx_path = "/content/drive/MyDrive/AiUniveristy/PROJECTS/Taxcom/20240705_Сценарии для ИИ.xlsx" # @param{type: 'string'}
key_word_collector = KeyWordsCollector(xlsx_path)

In [3]:
# @title Загрузка векторных баз (путь папке в которой множество баз данных - `multi_dbs`)
db_dir = '/content/drive/MyDrive/multi_dbs' # @param  ['/content/drive/MyDrive/multi_dbs','/content/drive/MyDrive/AiUniveristy/PROJECTS/Taxcom/multi_dbs']

# db_coordinator_path = "" # @param{type: 'string'}
embeddings = OpenAIEmbeddings(model='text-embedding-3-large')


doklines_db = FAISS.load_local(os.path.join(db_dir, '@Доклайнз'), embeddings, allow_dangerous_deserialization=True)
kkt_ofd_db = FAISS.load_local(os.path.join(db_dir, '@ККТ', 'ОФД'), embeddings, allow_dangerous_deserialization=True)
kkt_ofd_db_copywriten = FAISS.load_local(os.path.join(db_dir, '@ККТ_copywriten', 'ОФД'), embeddings, allow_dangerous_deserialization=True)
kripto_pro_db = FAISS.load_local(os.path.join(db_dir, '@КриптоПро ', ' КриптоАРМ'), embeddings, allow_dangerous_deserialization=True)
obshie_db = FAISS.load_local(os.path.join(db_dir, '@Общие'), embeddings, allow_dangerous_deserialization=True)
otchetnost_db = FAISS.load_local(os.path.join(db_dir, '@Отчётность'), embeddings, allow_dangerous_deserialization=True)
prochie_proekty_db = FAISS.load_local(os.path.join(db_dir, '@Прочие проекты'), embeddings, allow_dangerous_deserialization=True)
tokeny_db = FAISS.load_local(os.path.join(db_dir, '@Токены'), embeddings, allow_dangerous_deserialization=True)
uslugi_uc_db = FAISS.load_local(os.path.join(db_dir, '@Услуги УЦ'), embeddings, allow_dangerous_deserialization=True)
cto_db = FAISS.load_local(os.path.join(db_dir, '@ЦТО'), embeddings, allow_dangerous_deserialization=True)
odin_c_db = FAISS.load_local(os.path.join(db_dir, '@1С'), embeddings, allow_dangerous_deserialization=True)
def get_db(base_selector_answer:str, use_original_base_name=False):
    db=None
    if not use_original_base_name:
        base_selector_answer_clear = '@' + base_selector_answer.replace("['", "").replace("']","")
    else:
        base_selector_answer_clear = base_selector_answer

    if 'ЦТО' in base_selector_answer:
        return cto_db
    match base_selector_answer_clear:
        case '@Доклайнз':
            db = doklines_db
        case '@Отчётность':
            db = otchetnost_db
        case '@Общие':
            db = obshie_db
        case '@Прочие проекты':
            db = prochie_proekty_db
        case '@Услуги УЦ':
            db = uslugi_uc_db
        case '@ККТ/ОФД':
            db = kkt_ofd_db
        case '@ККТ/ОФД_copywriten':
            db = kkt_ofd_db_copywriten
        case '@КриптоПро / КриптоАРМ':
            db = kripto_pro_db
        case '@Токены':
            db = tokeny_db
        case '@ЦТО':
            db = cto_db
        case '@1С':
            db = odin_c_db
        case _:
            print(f'Неизвестная база данных, {base_selector_answer_clear}')
            raise ValueError(f'Неизвестная база данных, {base_selector_answer_clear}')
    return db

def find_most_relevant_chunks(question:str, base_selector_answer:str, chunks_n:int=5):
    db = get_db(base_selector_answer=base_selector_answer)
    chunks = db.similarity_search(query=question, k=chunks_n)
    return chunks


In [14]:
# @title Запрос пользователя
query = "Как экспортировать открытую часть сертификата?" # @param{type: 'string'}

In [ ]:
# @title Дораспределятор базы (pre_base_selector)
system_pre_base_selector = "Ты специалист тех. поддержки в компании которая занимается онлайн документооборотом. Клиент обращается к тебе с Запросом.Ты знаешь, что три категории Запросов клиентов: Прочие проекты, ЦТО и Иное. Ты знаешь, что все запросы относятся к категории Иное, кроме этих двух случаев: Ты строго следуешь правилам:  Правило 1 - Чтобы запрос относился к категории Прочие проекты в Запросе ОБЯЗАТЕЛЬНО должно быть дословно КАК МИНИМУМ ОДНО из этих словосочетаний дословно: 1) ВетИС; 2) ЕСИА; 3) Картотека документов, 4)Мобильное приложение Такском-Ветис; 5)Такском-Ветис; 6)Портал госуслуг; 7)Порядок обработки обращений; 8) Такском Управление доверенностями; 9) Такском-Ветис; 10) Такском-Визор; 11)Такском-Досье; 12) taxcom.ru; 13)Такском-Конвертер; 14)Такском-Ручка; 15)Такском-Управление Доверенностями; 16)ФГИС Меркурий; 17)Ветис.API; 18)Электронная доверенность (МЧД). Если ни одного из этих словосочетаний нет в тексте запроса ты знаешь что он  НЕ относится к категории Прочие проекты;Правило 2 - К категории ЦТО относятся ТОЛЬКО ИМЕННО эти десять Запросов(предложений) дословно СЛОВО В СЛОВО: 1)Какое назначение АИСТ?; 2) Что делать если в ПДО(Подтверждение даты отправки отчета) некорректное время формирования?; 3) Как внести товар или услугу в ККТ?: 4) Как передать ККТ на обслуживание?; 5)Как настроить/ использовать ККТ?; 6)Как модернизировать/обновить ККТ?; 7)Что делать если в распечатанном документе, драйвере или интерфейсе ККТ сообщение об ошибке?; 8)Что делать если в распечатанном чеке некорректные данные?; 9)Что делать если в распечатанном чеке нет данных?; 10)Как открыть/закрыть смену в ККТ?. Ты очень строго проверяешь Запрос на полное совпадение по тексту и если оно отличается, ты НЕ выбираешь ЦТО. Ты очень внимательно проверяешь, чтобы запрос был именно одним из этих десяти запросов, а не просто похожим на них." # @param{type: 'string'}
user_pre_base_selector = "Пожалуйста, сделай глубокий вдох и подумай шаг за шагом: 1)ВНИМАТЕЛЬНО проанализируй Запрос клиента; 2) Определи, к какой категории относится Запрос и напиши эту категорию в Ответ. Порядок отчета: python list, например:  ['ЦТО'] или ['Прочие проекты'] или ['Иное’']  и больше ничего. Запрос: {topic}." # @param{type: 'string'}
temp_pre_base_selector = 0 # @param {type: 'slider', min: 0, max: 2, step: 0.1}
model_pre_base_selector = 'gpt-4o' # @param  ['gpt-4o-mini','gpt-4o', 'gpt-4', 'gpt-3.5-turbo-16k-0613', 'gpt-3.5-turbo-1106']
save_to_google_table = True #@param{type: 'boolean'}



# Распределятор базы
async def pre_base_selector(system, user, topic, model, temperature):
    user = user.format(
        topic=topic,
    )
    messages = [
        {'role': 'system', 'content': system},
        {'role': 'user', 'content': user}
    ]
    completion = await client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    answer = completion.choices[0].message.content
    return answer

if '1с' in query.lower() or '1c' in query.lower():
     pre_base_selector_answer = "['1С']"
else:
    pre_base_selector_answer = await pre_base_selector(system_pre_base_selector, user_pre_base_selector, query, model_pre_base_selector, temp_pre_base_selector)


print_color('Запрос пользователя:', 'blue')
print(query)
print('-' * 50)
print_color(f'Ответ Дораспределятора базы:', 'pink')
print(pre_base_selector_answer)



if save_to_google_table:
    row_data = [
        current_datetime(),
        query,
        'Дораспределятор базы (pre_base_selector)',
        '',
        pre_base_selector_answer,
        system_pre_base_selector,
        user_pre_base_selector,
    ]
    gs_handler.append_row(row_data, 'Дораспределятор базы (pre_base_selector)')

Почему в отчете Кассы  в детализация отправленных чеков некорректные данные? Данные об отправке некорректные
--------------------------------------------------


['Иное']


In [ ]:
import nest_asyncio
import asyncio

nest_asyncio.apply()

# @title AUTO TEST - Дораспределятор базы (pre_base_selector)
async def get_similarity_score(base_selector_answer:str, question:str) -> float:
    db = get_db(base_selector_answer)
    chunk = await db.asimilarity_search_with_score(query=question, k=1)
    return chunk[0][1]

row_count = 109 # @param {"type":"integer"}

async def process_question(i, question, system_base_selector, user_base_selector, model_base_selector, temp_base_selector, rows_data, res):
    base_selector_answer = await pre_base_selector(system_base_selector, user_base_selector, question[0], model_base_selector, temp_base_selector)
    if 'Иное' in base_selector_answer:
        similarity_score = '/'
    else:
        similarity_score = await get_similarity_score(base_selector_answer, question[0])

    base_name = question[2]

    if base_name not in res:
        res[base_name] = {'scenario_count': 0, 'success_count': 0}

    res[base_name]['scenario_count'] += 1
    res[base_name]['success_count'] += 1

    is_correct = 'ДА'
    if ("['ЦТО']" in base_selector_answer and 'ЦТО' not in question[2]) or ("прочие проекты" in base_selector_answer.lower() and 'Прочие проекты' not in question[2]):
        is_correct = 'НЕТ'
        res[base_name]['success_count'] -= 1

    row_data = [
        i,
        question[0],
        'Дораспределятор базы (pre_base_selector)',
        '',
        base_selector_answer,
        system_base_selector,
        user_base_selector,
        question[2],
        question[1],
        is_correct,
        str(round(similarity_score, 2)) if similarity_score != '/' else similarity_score,
    ]

    rows_data.append(row_data)


async def main(auto_test_questions, system_base_selector, user_base_selector, model_base_selector, temp_base_selector):
    rows_data = [['Номер запроса', 'Запрос пользователя', 'Эксперт', 'Выбранный чанк', 'Ответ эксперта', 'system_promt', 'user_promt', 'Правильный ответ', 'id сценария', 'Ответ является правильным', 'Оценка\nсхожести']]
    res = {}
    success_count = 0

    tasks = [
        process_question(i, question, system_base_selector, user_base_selector, model_base_selector, temp_base_selector, rows_data, res)
        for i, question in enumerate(auto_test_questions, start=1)
    ]

    await asyncio.gather(*tasks)

    for base_name, stats in res.items():
        success_count += stats['success_count']

    return rows_data, res, success_count

# Usage example
auto_test_questions = gs_questions.read_data(f'A2:C{row_count}', 'Лист1')
rows_data, res, success_count = await main(auto_test_questions, system_pre_base_selector, user_pre_base_selector, model_pre_base_selector, temp_pre_base_selector)


rows_data = [rows_data[0]] + sorted(rows_data[1:], key=lambda x: x[0])

db_names = list(res.keys())
db_result = [f'{res[db_name]["success_count"]}/{res[db_name]["scenario_count"]:.2f} - ({res[db_name]["success_count"]/res[db_name]["scenario_count"]*100:.2f}%)' for db_name in db_names]
first_row = ['Дата и время', 'Количество запросов', 'процент успешно распределеных запросов', *db_names]
second_row = [current_datetime(), len(auto_test_questions), round(success_count/len(auto_test_questions), 2), *db_result]

rows_data.insert(0, second_row)
rows_data.insert(0, first_row)
rows_data.append([])


gs_questions.append_rows(rows_data, 'дораспределятор_auto_test')
print(f'Результаты выгружены в таблицу: {questions_spredsheet_url}')



In [ ]:
# @title Распределятор базы (base_selector)
system_base_selector = "Ты специалист тех. поддержки в компании которая занимается онлайн документооборотом. Клиент обращается к тебе с Запросом. Ты знаешь что Запрос - это вопрос по какому-то продукту или услуге компании, ошибки при подключении и использовании сервисами, формировании отчетов и заявок, взаимодействие с государственными органами и другие подобные вещи. Твой главный навык верно определить к какой из семи Категорий относится Запрос пользователя. Ты знаешь, что каждая Категория характеризуется набором признаков:  1) Доклайнз - включает запросы на тему: a) Подключение к сервисам; b)Регистрация в системах; c)Ошибки в работе продуктов; d)Вопросы по работе с мобильными приложениями; e) Отправка, сканирование и проверка документов; e)Вопросы по маркировке; f)Получение кодов; g)Консультации по продуктам; 2) ККТ/ОФД - ключает запросы на темы: a)Вопросы по работе с ОФД; b)Настройка и регистрация ККТ; c)Проблемы с фискальными данными; d)Вопросы по личному кабинету партнера ОФД; e)Ошибки в работе кассовой техники; f)Вопросы по визуализации данных; g)Регистрация и проверка чеков; h)Консультации по продуктам ОФД; i)Обновление и поддержка ПО для ККТ; 3) КриптоПро / КриптоАРМ -  включает запросы, связанные с криптографическими продуктами и услугами, такими как КриптоПро CSP и КриптоАРМ. Частые обращения касаются установки, настройки и использования криптографических сертификатов, а также устранения ошибок и проблем с лицензиями и контейнерами; 4) Общие -  включает запросы, связанные с управлением договорами и услугами, доступом к личным кабинетам, общими техническими вопросами и ошибками, а также консультациями по продуктам и услугам, которые не попадают в другие категории; 5)  Отчётность - включает запросы на темы: a) Подготовка и отправка отчетов; b)Настройка программного обеспечения для отчетности; c)Проблемы с отправкой документов; d) Ошибки при формировании отчетов; e)Взаимодействие с государственными органами; f) Вопросы по налоговой отчетности; g)Консультации по продуктам для отчетности; h)Проверка и исправление документов; i)Поддержка пользователей программ для отчетности; g) Интеграция с другими системами для отчетности; h)запросы со словом Доклайнер; 6)Токены - включает запросы, связанные с использованием и настройкой токенов, таких как Рутокен и ESMART. Частые обращения касаются проблем с установкой драйверов, ошибками при использовании токенов, настройкой и поддержкой криптографических сертификатов; 7) Услуги УЦ - включает запросы, связанные с использованием услуг удостоверяющих центров (УЦ). Частые обращения касаются проблем с входом на порталы, сертификатами, настройкой и поддержкой работы с государственными порталами. Пожалуйста, ВСЕГДА НА 100% следуй порядку отчета." # @param{type: 'string'}
user_base_selector = "Пожалуйста, сделай глубокий вдох и подумай шаг за шагом: ВНИМАТЕЛЬНО проанализируй Запрос клиента, на основании анализа напиши ОДНУ категорию, которая подходит под Запрос. Порядок отчета: python list, например: [‘Доклайнз’] или [‘ККТ/ОФД’] или [‘КриптоПро / КриптоАРМ’] или ['Общие'] или ['Отчётность'] или ['Токены']или ['Услуги УЦ'] и больше ничего. Запрос: {topic}." # @param{type: 'string'}
temp_base_selector = 0 # @param {type: 'slider', min: 0, max: 2, step: 0.1}
model_base_selector = 'gpt-4o' # @param  ['gpt-4o-mini','gpt-4o', 'gpt-4','gpt-3.5-turbo-16k-0613', 'gpt-3.5-turbo-1106']
save_to_google_table = True #@param{type: 'boolean'}



# Распределятор базы
async def base_selector(system, user, topic, model, temperature):
    if '1с' in topic.lower() or '1c' in topic.lower():
        return "['1С']"



    user = user.format(
        topic=topic,
    )
    messages = [
        {'role': 'system', 'content': system},
        {'role': 'user', 'content': user}
    ]
    completion = await client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    answer = completion.choices[0].message.content
    return answer


base_selector_answer = await base_selector(system_base_selector, user_base_selector, query, model_base_selector, temp_base_selector)


print_color('Запрос пользователя:', 'blue')
print(query)
print('-' * 50)
print_color(f'Ответ Распределятора базы:', 'pink')
print(base_selector_answer)



if save_to_google_table:
    row_data = [
        current_datetime(),
        query,
        'Распределятор базы (base_selector)',
        '',
        base_selector_answer,
        system_base_selector,
        user_base_selector,
    ]
    gs_handler.append_row(row_data, 'Распределятор базы (base_selector)')

Почему в отчете Кассы  в детализация отправленных чеков некорректные данные? Данные об отправке некорректные
--------------------------------------------------


['ККТ/ОФД']


In [ ]:
import nest_asyncio
import asyncio

nest_asyncio.apply()

# @title AUTO TEST - Распределятор базы (base_selector)
async def get_similarity_score(base_selector_answer:str, question:str) -> float:
    db = get_db(base_selector_answer)
    chunk = await db.asimilarity_search_with_score(query=question, k=1)
    return chunk[0][1]

async def get_score_with_chunk(base_selector_answer:str, question:str, use_original_base_name=False) -> float:
    db = get_db(base_selector_answer, use_original_base_name)
    chunk = await db.asimilarity_search_with_score(query=question, k=1)
    score = round(float(chunk[0][1]), 2)
    chunk_txt = chunk[0][0].page_content
    scenario_id = chunk[0][0].metadata['scenario_id']
    scenario_name = chunk[0][0].metadata['scenario_name']
    res = f"Результат\nсхожести: {score}\n Сценарий: {scenario_name}\nId сценаярия: {scenario_id}\nТекст выбранного чанка:\n{chunk_txt}"
    return res

row_count = 87 # @param {"type":"integer"}

async def process_question(i, question, system_base_selector, user_base_selector, model_base_selector, temp_base_selector, rows_data, res):
    base_selector_answer = await base_selector(system_base_selector, user_base_selector, question[0], model_base_selector, temp_base_selector)
    similarity_score_base_selector = await get_score_with_chunk(base_selector_answer, question[0])
    similarity_score_original = await get_score_with_chunk(question[2], question[0], True)

    row_data = [
        i,
        question[0],
        'Распределятор базы (base_selector)',
        base_selector_answer,
        system_base_selector,
        user_base_selector,
        question[2],
        question[1],
        'ДА' if base_selector_answer[2:-2] in question[2] else 'НЕТ',
        similarity_score_base_selector,
        similarity_score_original,
    ]

    rows_data.append(row_data)

    base_name = question[2]
    if base_name not in res:
        res[base_name] = {'scenario_count': 0, 'success_count': 0}

    res[base_name]['scenario_count'] += 1
    if base_selector_answer[2:-2] in base_name:
        res[base_name]['success_count'] += 1

async def main(auto_test_questions, system_base_selector, user_base_selector, model_base_selector, temp_base_selector):
    rows_data = [['Номер запроса', 'Запрос пользователя', 'Эксперт', 'Ответ эксперта', 'system_promt', 'user_promt', 'Правильный ответ', 'id сценария', 'Ответ является правильным', 'Оценка схожести чанка из\nбазы которую выбрал распределятор', 'Оценка схожести чанка из\n правильной базы']]
    res = {}
    success_count = 0

    tasks = [
        process_question(i, question, system_base_selector, user_base_selector, model_base_selector, temp_base_selector, rows_data, res)
        for i, question in enumerate(auto_test_questions, start=1)
    ]

    await asyncio.gather(*tasks)

    for base_name, stats in res.items():
        success_count += stats['success_count']

    return rows_data, res, success_count

# Usage example
auto_test_questions = gs_questions.read_data(f'A2:C{row_count}', 'Лист1')
rows_data, res, success_count = await main(auto_test_questions, system_base_selector, user_base_selector, model_base_selector, temp_base_selector)
rows_data = [rows_data[0]] + sorted(rows_data[1:], key=lambda x: x[0])

db_names = list(res.keys())
db_result = [f'{res[db_name]["success_count"]}/{res[db_name]["scenario_count"]:.2f} - ({res[db_name]["success_count"]/res[db_name]["scenario_count"]*100:.2f}%)' for db_name in db_names]
first_row = ['Дата и время', 'Количество запросов', 'процент успешно распределеных запросов', *db_names]
second_row = [current_datetime(), len(auto_test_questions), round(success_count/len(auto_test_questions), 2), *db_result]

rows_data.insert(0, second_row)
rows_data.insert(0, first_row)
rows_data.append([])

gs_questions.append_rows(rows_data, 'распределятор_auto_test')
print(f'Результаты выгружены в таблицу: {questions_spredsheet_url}')



Результаты выгружены в таблицу: https://docs.google.com/spreadsheets/d/1rm9bvjDFkRuyU94Lp1A5L4GlzUNkNt7nAA-mmWYf4TY/edit?usp=sharing


In [ ]:
# @title Дораспределятор + Распределятор (вместе) (total_base_selector)
async def total_base_selector(query:str):
    group = key_word_collector.find_relevant_group(query)
    if group:
        print(f"key_word_collector: {group}, formatted:['{group[1:]}']")
        formatted_group = f"['{group[1:]}']"
        return formatted_group

    if '1с' in query.lower() or '1c' in query.lower():
        pre_base_selector_answer = "['1С']"
    else:
        pre_base_selector_answer = await pre_base_selector(system_pre_base_selector, user_pre_base_selector, query, model_pre_base_selector, temp_pre_base_selector)

    if pre_base_selector_answer != "['Иное']":
        return pre_base_selector_answer

    base_selector_answer = await base_selector(system_base_selector, user_base_selector, query, model_base_selector, temp_base_selector)
    return base_selector_answer

total_base_selector_answer = await total_base_selector(query)


print_color('Запрос пользователя:', 'blue')
print(query)
print('-' * 50)
print_color(f'Ответ Распределятора базы (Дораспределятор + Распределятор):', 'pink')
print(total_base_selector_answer)



Почему в отчете Кассы  в детализация отправленных чеков некорректные данные? Данные об отправке некорректные
--------------------------------------------------


['ККТ/ОФД']


In [ ]:
import nest_asyncio
import asyncio

nest_asyncio.apply()

# @title AUTO TEST - Дораспределятор + Распределятор (вместе) (total_base_selector)
async def get_similarity_score(base_selector_answer:str, question:str) -> float:
    db = get_db(base_selector_answer)
    chunk = await db.asimilarity_search_with_score(query=question, k=1)
    return chunk[0][1]

async def get_score_with_chunk(base_selector_answer:str, question:str, use_original_base_name=False) -> float:
    db = get_db(base_selector_answer, use_original_base_name)
    chunk = await db.asimilarity_search_with_score(query=question, k=1)
    score = round(float(chunk[0][1]), 2)
    chunk_txt = chunk[0][0].page_content
    scenario_id = chunk[0][0].metadata['scenario_id']
    scenario_name = chunk[0][0].metadata['scenario_name']
    res = f"Результат\nсхожести: {score}\n Сценарий: {scenario_name}\nId сценаярия: {scenario_id}\nТекст выбранного чанка:\n{chunk_txt}"
    return res

row_count = 123 # @param {"type":"integer"}

async def process_question(i, question, system_base_selector, user_base_selector, model_base_selector, temp_base_selector, rows_data, res):
    base_selector_answer = await total_base_selector(question[0])
    similarity_score_base_selector = await get_score_with_chunk(base_selector_answer, question[0])
    similarity_score_original = await get_score_with_chunk(question[2], question[0], True)

    row_data = [
        i,
        question[0],
        'Дораспределятор + Распределятор базы (base_selector)',
        base_selector_answer,
        system_base_selector,
        user_base_selector,
        question[2],
        question[1],
        'ДА' if base_selector_answer[2:-2] in question[2] else 'НЕТ',
        similarity_score_base_selector,
        similarity_score_original,
    ]

    rows_data.append(row_data)

    base_name = question[2]
    if base_name not in res:
        res[base_name] = {'scenario_count': 0, 'success_count': 0}

    res[base_name]['scenario_count'] += 1
    if base_selector_answer[2:-2] in base_name:
        res[base_name]['success_count'] += 1

async def main(auto_test_questions, system_base_selector, user_base_selector, model_base_selector, temp_base_selector):
    rows_data = [['Номер запроса', 'Запрос пользователя', 'Эксперт', 'Ответ эксперта', 'system_promt', 'user_promt', 'Правильный ответ', 'id сценария', 'Ответ является правильным', 'Оценка схожести чанка из\nбазы которую выбрал распределятор', 'Оценка схожести чанка из\n правильной базы']]
    res = {}
    success_count = 0

    tasks = [
        process_question(i, question, system_base_selector, user_base_selector, model_base_selector, temp_base_selector, rows_data, res)
        for i, question in enumerate(auto_test_questions, start=1)
    ]

    await asyncio.gather(*tasks)

    for base_name, stats in res.items():
        success_count += stats['success_count']

    return rows_data, res, success_count

# Usage example
auto_test_questions = gs_questions.read_data(f'A2:C{row_count}', 'Лист1')
rows_data, res, success_count = await main(auto_test_questions, system_base_selector, user_base_selector, model_base_selector, temp_base_selector)
rows_data = [rows_data[0]] + sorted(rows_data[1:], key=lambda x: x[0])

db_names = list(res.keys())
db_result = [f'{res[db_name]["success_count"]}/{res[db_name]["scenario_count"]:.2f} - ({res[db_name]["success_count"]/res[db_name]["scenario_count"]*100:.2f}%)' for db_name in db_names]
first_row = ['Дата и время', 'Количество запросов', 'процент успешно распределеных запросов', *db_names]
second_row = [current_datetime(), len(auto_test_questions), round(success_count/len(auto_test_questions), 2), *db_result]

rows_data.insert(0, second_row)
rows_data.insert(0, first_row)
rows_data.append([])

gs_questions.append_rows(rows_data, 'распределятор_auto_test')
print(f'Результаты выгружены в таблицу: {questions_spredsheet_url}')



key_word_collector: @Доклайнз, formatted:['Доклайнз']
key_word_collector: @Отчётность, formatted:['Отчётность']
key_word_collector: @Прочие проекты, formatted:['Прочие проекты']
key_word_collector: @Общие, formatted:['Общие']
key_word_collector: @Общие, formatted:['Общие']
key_word_collector: @Отчётность, formatted:['Отчётность']
key_word_collector: @Отчётность, formatted:['Отчётность']
key_word_collector: @Токены, formatted:['Токены']
key_word_collector: @Токены, formatted:['Токены']
key_word_collector: @Услуги УЦ, formatted:['Услуги УЦ']
key_word_collector: @Услуги УЦ, formatted:['Услуги УЦ']
key_word_collector: @Прочие проекты, formatted:['Прочие проекты']
key_word_collector: @Услуги УЦ, formatted:['Услуги УЦ']
key_word_collector: @Прочие проекты, formatted:['Прочие проекты']
key_word_collector: @Прочие проекты, formatted:['Прочие проекты']
key_word_collector: @1С, formatted:['1С']
key_word_collector: @1С, formatted:['1С']
key_word_collector: @1С, formatted:['1С']
key_word_collector

In [15]:
#@title Ответ распределятора вручную

base_selector_answer = 'КриптоПро / КриптоАРМ' # @param  ['Доклайнз','Отчётность','Общие', 'Прочие проекты', 'Услуги УЦ', 'ККТ/ОФД', 'КриптоПро / КриптоАРМ', 'Токены', 'ЦТО', '1С']

In [16]:
# @title Поиск самых релевантных чанков по запросу (на основании ответа распределятора)
db = None
question = query
chunks_n = 5 # @param{type: 'integer'}
save_to_google_table = True #@param{type: 'boolean'}


chunks = find_most_relevant_chunks(
    question=question,
    base_selector_answer=base_selector_answer,
    chunks_n=chunks_n
)

print_color("Запрос пользователя:", color='green')
print(question, end='\n\n')


print_color("Самые релевантные чанки: ", color='green')
chunk_cols_data = []

for i, chunk in enumerate(chunks, start=1):
    part_1 = f'Чанк: #{i}'
    part_2 = f'Имя сценария: #{chunk.metadata.get("scenario_name")}'
    part_3 = f'Тэги сценария: #{chunk.metadata.get("scenario_tags")}'
    part_4 = f'Темплейт сценария: #{chunk.metadata.get("scenario_template")}'
    part_5 = f'Содержимое сценария \n {chunk.page_content}'

    print_color(part_1, color='blue')
    print_color(part_2, color='blue')
    print_color(part_3, color='green')
    print_color(part_4, color='orange')
    print(part_5)
    print('\n\n')
    chunk_col_data = part_1 + '\n' + part_2 + '\n' + part_3 + '\n' + part_4 + '\n' + part_5
    chunk_cols_data.append(chunk_col_data)


if save_to_google_table:
    row_data = [
        current_datetime(),
        base_selector_answer,
        question,
        *chunk_cols_data,
    ]
    gs_handler.append_row(row_data, 'Гипотез3 (самые релевантные чанки)')
    print_color("Данные успешно сохранены в Google Drive.", color='green')
    print(f'url: https://docs.google.com/spreadsheets/d/1607FPIz2-SOwR31rPYlFkYrOo37q9rC9xixcoEo5G8E/edit?usp=sharing')

Как экспортировать открытую часть сертификата?



Содержимое сценария 
 Подробная информация есть в статье БЗ «Статья: Как экспортировать открытую часть сертификата». Статья: "Как экспортировать открытую часть сертификата":
 
Как экспортировать открытую часть сертификата — База знаний
Как экспортировать открытую часть сертификата
Материал из База знаний
		Перейти к:		навигация, 		поиск
Развернуть/свернуть всё
Что такое открытая часть сертификата
Открытая часть сертификата представляется в виде электронного сертификата или бумажного документа. Такой сертификат отображается в виде файла с разрешением .cer.Открытый ключ используется для:
Проверки подписи под документом. Подписание возможно только при наличии ключевой пары.
Работы с партнерами, контрагенты могут использовать его для шифрования данных, которые отправляют владельцу. Расшифровать эти данные после отправки получится только при наличии закрытой части сертификата.
Доступ к закрытому ключу имеет только владелец сертификата, открытая часть доступна всем участникам электронного вз

Содержимое сценария 
 Необходимо:  Загрузить открытую часть сертификата из «Статья: Реестра сертификатов[https://cert.taxcom.ru/Private.aspx]» и установить ее через «КриптоПро». Вход по логину: TaxcomUser и паролю: Taxcomcertificate (доступ только для сотрудников).    Зайти на Статья: сайт[http://cert.taxcom.ru/Private.aspx], вход по логину: TaxcomUser и паролю: Taxcomcertificate (доступ только для сотрудников).  Найти сертификат по его серийному номеру или отпечатку.  Нажать «Показать сертификат…».  Перейти в «КриптоПро»:     Windows 7: «Пуск» - «Все программы» - «КРИПТО-ПРО» - «КриптоПро CSP»;   Windows 8: нажать ПКМ на рабочем столе - выбрать внизу экрана «Все приложения» - «КРИПТО-ПРО» - «КриптоПро CSP»;   Windows 10: «Пуск» - «КРИПТО-ПРО» - «КриптоПро CSP».      На вкладке «Сервис» нажать «Установить личный сертификат…».  В открывшемся окне нажать «Обзор», указать путь до файла с открытой частью сертификата и нажать «Открыть».  Нажать «Далее», проверить данные в сертификате и еще 

Содержимое сценария 
 В «КриптоПро CSP» версии 5.0 сертификат отображается в поле «Неэкспортируемые контейнеры», если он является некопируемым. Данный параметр определяется при получении сертификата.  Если Клиент хочет скопировать сертификат, указанный в этом поле, необходимо сообщить, что возможность скопировать его на другой носитель или в реестр отсутствует. Использовать данный сертификат (в том числе сертификат УЦ ФНС) можно только на токене, на который он был выпущен.  Отсутствует возможность копировать сертификаты, выпущенные на:    «Рутокен ЭЦП 2.0»;  «Рутокен ЭЦП 2.0 Flash»;  «Рутокен ЭЦП 3.0»;  «Рутокен ЭЦП PKI»;  «Рутокен 2151»;  JaCarta 2SE;  JaCarta 2ГОСТ;  JaCarta SF/ГОСТ;  JaCarta PKI;  Etoken Gost.  





Содержимое сценария 
 Необходимо:    Открыть «Пуск» - «Все программы» - «КриптоПро» - «КриптоПро PKI»;  В разделе «Управление лицензиями» правой кнопкой мыши нажать на «TSP (OCSP) Client»;  Выбрать «Все задачи» ­- «Ввести серийный номер»;  Ввести серийный номер и нажать «ОК».  





Содержимое сценария 
 Необходимо проконсультировать Клиента по статье Базы знаний «Статья: В списке доступных контейнеров пусто или не отображаются контейнеры на носителе». Статья: "В списке доступных контейнеров пусто или не отображаются контейнеры на носителе":
 
В списке доступных контейнеров пусто или не отображаются контейнеры на носителе — База знаний
В списке доступных контейнеров пусто или не отображаются контейнеры на носителе
Материал из База знаний
		Перейти к:		навигация, 		поиск
Развернуть/свернуть всё
1. Необходимо уточнить у Клиента, где находится сертификат;
2. Если сертификат находится:
 В реестре 
Необходимо:
1. Убедиться, что установлена «КриптоПро CSP» сертифицированной версии и ее лицензия действительна.
Если установлена «КриптоПро CSP» несертифицированной версии, необходимо переустановить на сертифицированную совместимую версию:
ОС Windows, для Windows 11 требуется версия 5.0 R2 (12000) или выше;
ОС Linux, требуется версия 5.0 или выше.
2. Если сертификат выпущен 

url: https://docs.google.com/spreadsheets/d/1607FPIz2-SOwR31rPYlFkYrOo37q9rC9xixcoEo5G8E/edit?usp=sharing


In [19]:
# @title Специалист по выбору самого релевантного сценария (scenario_selecter)
question = "Как экспортировать открытую часть сертификата?" # @param{type: 'string'}
system_scenario_selecter = "Ты специалист в компании, которая занимается документооборотом. К тебе обращается пользователь с Запросом. У тебя есть Топ 5 Инструкций, которые относятся к запросу пользователя. Твоя задача: выбрать ОДНУ инструкцию, которая наиболее релевантна к запросу пользователя. Ты на 100% следуешь ПРАВИЛАМ: 1. Ты отбираешь ТОЛЬКО ОДНУ инструкцию, которая  точно имеет информацию по Запросу пользователя; 2. Ты пишешь полное содержание отобранной тобой Инструкции слово в слово." # @param{type: 'string'}
user_scenario_selecter = "Пожалуйста, действуй шаг за шагом: 1. Внимательно проанализируй и пойми весь смысл Запроса пользователя: {topic}; 2. Внимательно проанализируй предоставленные тебе Топ 5 Инструкций; 3. Напиши в своем ответе только НОМЕР САМОЙ РЕЛЕВАНТНОЙ инструкции и БОЛЬШЕ НИЧЕГО. {message_content}. Топ 5 Инструкций: {message_content} Запрос пользователя: {topic}" # @param{type: 'string'}
temp_scenario_selecter = 0 # @param {type: 'slider', min: 0, max: 2, step: 0.1}
model_scenario_selecter = 'gpt-4o-mini' # @param  ['gpt-4o-mini','gpt-4o', 'gpt-4', 'gpt-3.5-turbo-16k-0613', 'gpt-3.5-turbo-1106']

save_to_google_table = True #@param{type: 'boolean'}

db = get_db(base_selector_answer)
most_relevant_chunks = db.similarity_search(query=query, k=5)
message_content = [f'Сценарий №{i}\n{chunk.page_content}\n\n' for i, chunk in enumerate(most_relevant_chunks, start=1)]
message_content = '\n'.join(message_content)


async def scenario_selecter(system, user, topic, message_content, model, temperature):
    user = user.format(
        topic=topic,
        message_content=message_content
    )
    messages = [
        {'role': 'system', 'content': system},
        {'role': 'user', 'content': user}
    ]
    completion = await client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    answer = completion.choices[0].message.content
    return answer

scenario_selecter_answer = await scenario_selecter(system_scenario_selecter, user_scenario_selecter, question, message_content, model_scenario_selecter, temp_scenario_selecter)


print_color('Отобранный чанк:', 'blue')
print(message_content)
print_color('Запрос пользователя:', 'blue')
print(question)
print('-' * 50)
print_color(f'Специалист по выбору самого релевантного сценария (scenario_selecter):', 'pink')
print(scenario_selecter_answer)



if save_to_google_table:
    row_data = [
        current_datetime(),
        question,
        'Копирайтер Яны (scenario_selecter)',
        message_content,
        scenario_selecter_answer,
        system_scenario_selecter,
        user_scenario_selecter,
    ]
    gs_handler.append_row(row_data, 'Специалист по выбору самого релевантного сценария')
    print_color("Данные успешно сохранены в Google Drive.", color='green')
    print(f'url: https://docs.google.com/spreadsheets/d/1607FPIz2-SOwR31rPYlFkYrOo37q9rC9xixcoEo5G8E/edit?usp=sharing')


Сценарий №1
Подробная информация есть в статье БЗ «Статья: Как экспортировать открытую часть сертификата». Статья: "Как экспортировать открытую часть сертификата":
 
Как экспортировать открытую часть сертификата — База знаний
Как экспортировать открытую часть сертификата
Материал из База знаний
		Перейти к:		навигация, 		поиск
Развернуть/свернуть всё
Что такое открытая часть сертификата
Открытая часть сертификата представляется в виде электронного сертификата или бумажного документа. Такой сертификат отображается в виде файла с разрешением .cer.Открытый ключ используется для:
Проверки подписи под документом. Подписание возможно только при наличии ключевой пары.
Работы с партнерами, контрагенты могут использовать его для шифрования данных, которые отправляют владельцу. Расшифровать эти данные после отправки получится только при наличии закрытой части сертификата.
Доступ к закрытому ключу имеет только владелец сертификата, открытая часть доступна всем участникам электронного взаимодейств

Как экспортировать открытую часть сертификата?
--------------------------------------------------


1


url: https://docs.google.com/spreadsheets/d/1607FPIz2-SOwR31rPYlFkYrOo37q9rC9xixcoEo5G8E/edit?usp=sharing


In [10]:
# @title Копирайтер Яны (yana_copywriter)
system_yana_copywriter = "Ты прекрасный копирайтер. Тебе дают текст Инструкции для консультанта компании занимающейся онлайн документооборотом. Твоя задача взять текст инструкции и НИЧЕГО в нем НЕ МЕНЯТЬ А ПРОСТО отредактировать его так, чтобы получился последовательный и удобный для чтения Ответ. Ты всегда следуешь правилам: а) Ты знаешь что каждое предложение в Инструкции важное и его нужно сохранить; b) Ты оставляешь ВЕСЬ текст Инструкции который редактируешь, просто структурируешь эту информацию в своем Ответе; c) Ты всегда оставляешь все ссылки из Инструкции в тексте Ответа; d) Ты мастерски делишь текст на абзацы по смыслу; e) ты не пишешь слово 'ответ' в начале ответа. " # @param{type: 'string'}
user_yana_copywriter = "Пожалуйста, сделай глубокий вдох и действуй шаг за шагом: 1) Проанализируй Инструкцию ; 2) Напиши понятный и структурированный Ответ следуя всем правилам. Инструкция: {message_content}" # @param{type: 'string'}
temp_yana_copywriter = 0 # @param {type: 'slider', min: 0, max: 2, step: 0.1}
model_yana_copywriter = 'gpt-4o-mini' # @param  ['gpt-4o-mini','gpt-4o', 'gpt-4', 'gpt-3.5-turbo-16k-0613', 'gpt-3.5-turbo-1106']

chunk_number = "1" # @param  [1, 2, 3, 4, 5]
save_to_google_table = True #@param{type: 'boolean'}

db = get_db(base_selector_answer)
most_relevant_chunk = db.similarity_search(query=query, k=5)[int(chunk_number)-1]
message_content = most_relevant_chunk.page_content
# Преподаватель
async def yana_copywriter(system, user, topic, message_content, model, temperature):
    user = user.format(
        topic=topic,
        message_content=message_content
    )
    messages = [
        {'role': 'system', 'content': system},
        {'role': 'user', 'content': user}
    ]
    completion = await client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    answer = completion.choices[0].message.content
    return answer

copywriter_answer = await yana_copywriter(system_yana_copywriter, user_yana_copywriter, question, message_content, model_yana_copywriter, temp_yana_copywriter)


print_color('Отобранный чанк:', 'blue')
print(message_content)
print_color('Запрос пользователя:', 'blue')
print(question)
print('-' * 50)
print_color(f'Ответ тех поддержки:', 'pink')
print(copywriter_answer)



if save_to_google_table:
    row_data = [
        current_datetime(),
        question,
        'Копирайтер Яны (yana_copywriter)',
        message_content,
        copywriter_answer,
        system_yana_copywriter,
        user_yana_copywriter,
    ]
    gs_handler.append_row(row_data, 'Копирайтер Яны (yana_copywriter)')
    print_color("Данные успешно сохранены в Google Drive.", color='green')
    print(f'url: https://docs.google.com/spreadsheets/d/1607FPIz2-SOwR31rPYlFkYrOo37q9rC9xixcoEo5G8E/edit?usp=sharing')


Полный текст сообщения:   «Дополнительная информация: Revocation request is not available for roaming operator ***».  Необходимо:  1. Сообщить Клиенту, что возможность аннулировать документ в роуминге сейчас есть только с операторами:    «НТЦ СТЭК» (2BA);  «НИИАС» (2JD);  «Эвотор ОФД» (2VO);  «Русь - Телеком» (2AD);  «Оператор-ЦРПТ» (2LT);  «СИСЛИНК» (2JM);  «ЭТП ГПБ» (2LB);  «Бифит ЭДО» (2LG);  «Мостинфо-Екатеринбург» (2CA);  «Калуга-Астрал» (2AE) и СКБ «Контур» (2BM), если документооборот завершен;  «Тензор» (2BE), если содержит только один файл вложениях.   Если Клиенту нужно аннулировать документ с другим оператором в роуминге, необходимо уточнить, есть ли в документе информация по маркированной продукции. Если:    Есть, рекомендовать:     Аннулировать документы неформализованным документом:           «Статья: [Такском-Файлер] Как сформировать и отправить неформализованный документ[]»;            «Статья: [Модуль для 1С] Как создать и отправить неформализованный документ[]»;       

Почему в отчете Кассы  в детализация отправленных чеков некорректные данные? Данные об отправке некорректные
--------------------------------------------------


Дополнительная информация: Revocation request is not available for roaming operator ***.

Необходимо:

1. Сообщить Клиенту, что возможность аннулировать документ в роуминге сейчас доступна только с операторами:
   - «НТЦ СТЭК» (2BA);
   - «НИИАС» (2JD);
   - «Эвотор ОФД» (2VO);
   - «Русь - Телеком» (2AD);
   - «Оператор-ЦРПТ» (2LT);
   - «СИСЛИНК» (2JM);
   - «ЭТП ГПБ» (2LB);
   - «Бифит ЭДО» (2LG);
   - «Мостинфо-Екатеринбург» (2CA);
   - «Калуга-Астрал» (2AE);
   - СКБ «Контур» (2BM), если документооборот завершен;
   - «Тензор» (2BE), если содержит только один файл в вложениях.

2. Если Клиенту нужно аннулировать документ с другим оператором в роуминге, необходимо уточнить, есть ли в документе информация по маркированной продукции.

   - Если информация есть, рекомендовать:
     - Аннулировать документы неформализованным документом:
       - «Статья: [Такском-Файлер] Как сформировать и отправить неформализованный документ[]»;
       - «Статья: [Модуль для 1С] Как создать и отправит

url: https://docs.google.com/spreadsheets/d/1607FPIz2-SOwR31rPYlFkYrOo37q9rC9xixcoEo5G8E/edit?usp=sharing


In [13]:
# @title Помощник тех поддержки (tech_support)
question = "Как отозвать отправленное gредложение об аннулировании роуминга? Я не знаю, какой у меня сейчас оператор" # @param{type: 'string'}
chunk = "" # @param{type: 'string'}
system_tech_support = "Ты специалист в компании, которая занимается документооборотом. К тебе обращается клиент с Запросом. Ты значешь что в ответ на Запрос пользователя ты выводишь ему текст Инструкции. Твоя главная задача проанализировать Запрос пользователя и Инструкцию и определить, есть ли в Запросе пользователя Уточняющая информация. Ты знаешь, что Уточняющая информация - это информация, которая уточняет запрос пользователя и помогает определить, какие пункты в Инструкции являются наиболее релевантными. Например: Инструкция состоит из нескольких пунктов. В одном из пунктов могут быть разные действия если обращается клиент либо партнер, или разные действия в зависимости от того какой браузер использует клиент, также могут быть разные действия в зависимости от необходимого результата и подобное. Если в запросе пользователя есть Уточняющая информация, при выводе инструкции ты пишешь все пункт за пунктом, но в пункте где есть уточнение выводишь только данные релевантные запрусу пользователя и уточняющей информации .Твоя главная задача - 1) проанализировать запрос пользователя и инструкцию чтобы  понять, есть ли в запросе пользователя уточняющая информация; 2) вывести в ответ текст инструкции учитывая уточняющую информацию или ЕСЛИ УТОЧНЯЮЩЕЙ ИНФОРМАЦИИ НЕТ ты выводишь ВЕСЬ ТЕКСТ ИНСТРУКЦИИ." # @param{type: 'string'}
user_tech_support = "Пожалуйста, сделай глубокий вдох и действуй шаг за шагом: 1) Проанализируй запрос пользователя и инструкцию; 2) Определи, есть ли в запросе пользователя УТОЧНЯЮЩАЯ информация; 2.1) если в запросе пользователя есть уточняющая информация выведи в ответ текст инструкции с учетом уточняющей информации; 2.2)  если в запросе пользователя нет уточняющей информации ты выводишь в ответ всю Инструкцию ЦЕЛИКОМ. Напиши текст Ответа  И БОЛЬШЕ НИЧЕГО. Инструкция: {message_content}. Запрос: {topic}" # @param{type: 'string'}
temp_tech_support = 0 # @param {type: 'slider', min: 0, max: 2, step: 0.1}
model_tech_support = 'gpt-4o-mini' # @param  ['gpt-4o-mini','gpt-4o', 'gpt-4', 'gpt-3.5-turbo-16k-0613', 'gpt-3.5-turbo-1106']

save_to_google_table = True #@param{type: 'boolean'}

db = get_db(base_selector_answer)
# most_relevant_chunk = db.similarity_search(query=query, k=5)[int(chunk_number)-1]
if not chunk:
    message_content = copywriter_answer
else:
    message_content = chunk
# Преподаватель
async def tech_support(system, user, topic, message_content, model, temperature):
    user = user.format(
        topic=topic,
        message_content=message_content
    )
    messages = [
        {'role': 'system', 'content': system},
        {'role': 'user', 'content': user}
    ]
    completion = await client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    answer = completion.choices[0].message.content
    return answer

tech_support_answer = await tech_support(system_tech_support, user_tech_support, question, message_content, model_tech_support, temp_tech_support)


print_color('Отобранный чанк:', 'blue')
print(message_content)
print_color('Запрос пользователя:', 'blue')
print(question)
print('-' * 50)
print_color(f'Ответ помощника тех поддержки:', 'pink')
print(tech_support_answer)



if save_to_google_table:
    row_data = [
        current_datetime(),
        question,
        'Помощник тех поддержки (tech_support)',
        message_content,
        tech_support_answer,
        system_tech_support,
        user_tech_support,
    ]
    gs_handler.append_row(row_data, 'Помощник тех поддержки (tech_support)')
    print_color("Данные успешно сохранены в Google Drive.", color='green')
    print(f'url: https://docs.google.com/spreadsheets/d/1607FPIz2-SOwR31rPYlFkYrOo37q9rC9xixcoEo5G8E/edit?usp=sharing')


Дополнительная информация: Revocation request is not available for roaming operator ***.

Необходимо:

1. Сообщить Клиенту, что возможность аннулировать документ в роуминге сейчас доступна только с операторами:
   - «НТЦ СТЭК» (2BA);
   - «НИИАС» (2JD);
   - «Эвотор ОФД» (2VO);
   - «Русь - Телеком» (2AD);
   - «Оператор-ЦРПТ» (2LT);
   - «СИСЛИНК» (2JM);
   - «ЭТП ГПБ» (2LB);
   - «Бифит ЭДО» (2LG);
   - «Мостинфо-Екатеринбург» (2CA);
   - «Калуга-Астрал» (2AE);
   - СКБ «Контур» (2BM), если документооборот завершен;
   - «Тензор» (2BE), если содержит только один файл в вложениях.

2. Если Клиенту нужно аннулировать документ с другим оператором в роуминге, необходимо уточнить, есть ли в документе информация по маркированной продукции.

   - Если информация есть, рекомендовать:
     - Аннулировать документы неформализованным документом:
       - «Статья: [Такском-Файлер] Как сформировать и отправить неформализованный документ[]»;
       - «Статья: [Модуль для 1С] Как создать и отправит

Как отозвать отправленное gредложение об аннулировании роуминга? Я не знаю, какой у меня сейчас оператор
--------------------------------------------------


Весь текст инструкции целиком:

Дополнительная информация: Revocation request is not available for roaming operator ***.

Необходимо:

1. Сообщить Клиенту, что возможность аннулировать документ в роуминге сейчас доступна только с операторами:
   - «НТЦ СТЭК» (2BA);
   - «НИИАС» (2JD);
   - «Эвотор ОФД» (2VO);
   - «Русь - Телеком» (2AD);
   - «Оператор-ЦРПТ» (2LT);
   - «СИСЛИНК» (2JM);
   - «ЭТП ГПБ» (2LB);
   - «Бифит ЭДО» (2LG);
   - «Мостинфо-Екатеринбург» (2CA);
   - «Калуга-Астрал» (2AE);
   - СКБ «Контур» (2BM), если документооборот завершен;
   - «Тензор» (2BE), если содержит только один файл в вложениях.

2. Если Клиенту нужно аннулировать документ с другим оператором в роуминге, необходимо уточнить, есть ли в документе информация по маркированной продукции.

   - Если информация есть, рекомендовать:
     - Аннулировать документы неформализованным документом:
       - «Статья: [Такском-Файлер] Как сформировать и отправить неформализованный документ[]»;
       - «Статья: [Модул

url: https://docs.google.com/spreadsheets/d/1607FPIz2-SOwR31rPYlFkYrOo37q9rC9xixcoEo5G8E/edit?usp=sharing
